In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

from vqvaes.vq import VQ
import torch

In [25]:
vq = VQ(
    codebook_size=4,
    codebook_dim=12
)

vq

VQ(
  (codebook): Embedding(4, 12)
)

In [32]:
encoding = torch.randn((1,  2, 2, 12))
vq(encoding)

{'quantized': tensor([[[[ 0.0660, -1.2311, -0.7915, -0.6858, -0.4377, -0.7634,  1.1600,
            -0.5719, -1.2919, -0.7813,  0.0328,  0.1664],
           [ 0.0660, -1.2311, -0.7915, -0.6858, -0.4377, -0.7634,  1.1600,
            -0.5719, -1.2919, -0.7813,  0.0328,  0.1664]],
 
          [[ 0.8181,  0.5823,  1.0346, -0.0613,  1.5635,  1.0844, -0.4846,
            -0.3286,  1.2524,  0.5251,  0.2822, -0.2040],
           [-0.4393, -0.0342,  1.4439, -0.0670, -1.8643, -0.7695,  0.5740,
            -0.7604,  1.3218, -2.2603, -0.0298, -1.3140]]]],
        grad_fn=<ViewBackward0>)}

In [15]:
a = torch.tensor([[1, 2]])
b = torch.tensor([[3, 4],
                  [5, 6]])
a.shape, b.shape

(torch.Size([1, 2]), torch.Size([2, 2]))

In [9]:
c = a - b
c

tensor([[-2, -2],
        [-4, -4]])